In [1]:
# %load prsent_2
import numpy as np
import matplotlib.pyplot as plt
import os
import datetime
import netCDF4
import pandas as pd

data_dir = 'data_GOES'

# Load abundance data
datafile_abundance = 'Mg_summed_ahead_2023_1hr_level1_11.txt'
data_abundance = np.loadtxt(datafile_abundance, skiprows=35)
yy_abundance = data_abundance[:,0]
day_yy_abundance = data_abundance[:,1]
HH_abundance = data_abundance[:,2]
MM_abundance = data_abundance[:,3]
SS_abundance = data_abundance[:,4]

start_date_abundance = datetime.datetime.strptime('2023-01-01T00:00:00','%Y-%m-%dT%H:%M:%S')

time_abundance = []
for i in range(len(yy_abundance)):
    date = datetime.datetime.strptime(str(int(yy_abundance[i]))+'-'+str(int(day_yy_abundance[i]))+'-'+str(int(HH_abundance[i]))+':'+str(int(MM_abundance[i]))+':'+str(int(SS_abundance[i])), '%Y-%j-%H:%M:%S')
    tim_ = (date - start_date_abundance).total_seconds()
    time_abundance.append(tim_)
time_abundance = np.array(time_abundance)

# Load GOES data
goes_files = [f for f in os.listdir(data_dir) if f.endswith('.nc')]



## Calculate abundance for mg

da = data_abundance[:,1+5]
datafile_H = 'H_summed_ahead_2023_1hr_level1_11.txt'
data_H = np.loadtxt(datafile_H, skiprows=36)
da_H = data_H[:,1+5]
abundance = 12 + np.log10(da/da_H)

# Error calculation
da_error = data_abundance[:,14]
da_H_error = data_H[:,14]
sigma = np.where((da != 0) & (da_H != 0), 0.432944*np.sqrt((da_error/da)**2 + (da_H_error/da_H)**2), np.nan)


## calclation abumndance for Al

datafile_Fe = 'Fe_summed_ahead_2023_1hr_level1_11.txt'
data_abundance_Fe = np.loadtxt(datafile_Fe,skiprows=36)
da_Fe = data_abundance_Fe[:,1+5]
abundance_Fe = 12 + np.log10(da_Fe/da_H)
da_Fe_error = data_abundance_Fe[:,16]
sigma_Fe = np.where((da_Fe != 0) & (da_H != 0), 0.432944*np.sqrt((da_Fe_error/da_Fe)**2 + (da_H_error/da_H)**2), np.nan)


## calclation abumndance for O

datafile_O = 'O_summed_ahead_2023_1hr_level1_11.txt'
data_abundance_O = np.loadtxt(datafile_O, skiprows=35) 
da_O = data_abundance_O[:,1+5]
abundance_O = 12 + np.log10(da_O/da_H)
da_O_error = data_abundance_O[:,14]
sigma_O = np.where((da_O != 0) & (da_H != 0), 0.432944*np.sqrt((da_O_error/da_O)**2 + (da_H_error/da_H)**2), np.nan)


## calclation abumndance for Si

datafile_Si = 'Si_summed_ahead_2023_1hr_level1_11.txt'
data_abundance_Si = np.loadtxt(datafile_Si, skiprows=36)
da_Si = data_abundance_Si[:,1+5]
abundance_Si = 12 + np.log10(da_Si/da_H)
da_Si_error = data_abundance_Si[:,16]
sigma_Si = np.where((da_Si != 0) & (da_H != 0), 0.432944*np.sqrt((da_Si_error/da_Si)**2 + (da_H_error/da_H)**2), np.nan)


## calclation abumndance for He

datafile_He = 'He_summed_ahead_2023_1hr_level1_11.txt'
data_abundance_He = np.loadtxt(datafile_He, skiprows=34)
da_He = data_abundance_He[:,1+5]
abundance_He = 12 + np.log10(da_He/da_H)
da_He_error = data_abundance_He[:,12]
sigma_He = np.where((da_He != 0) & (da_H != 0), 0.432944*np.sqrt((da_He_error/da_He)**2 + (da_H_error/da_H)**2), np.nan)


fig, axs = plt.subplots(6, figsize=(10, 12))

axs[0].errorbar((time_abundance/(3600*24)), abundance, sigma, fmt='o', capsize=5, label='Abundance with Error', color='blue')
axs[0].set_ylabel('Abundance_mag')
axs[0].set_xlabel('Time (days)')
#axs[0].legend(loc='upper left')                    


axs[1].errorbar((time_abundance/(3600*24)), abundance_Fe, sigma_Fe, fmt='o', capsize=5, label='Abundance with Error', color='red')
axs[1].set_ylabel('Abundance_Fe')
axs[1].set_xlabel('Time (days)')
#axs[1].legend(loc='upper left')                    

axs[2].errorbar((time_abundance/(3600*24)), abundance_O, sigma_O, fmt='o', capsize=5, label='Abundance with Error', color='green')
axs[2].set_ylabel('Abundance_O')
axs[2].set_xlabel('Time (days)')
#axs[2].legend(loc='upper left')                    

axs[3].errorbar((time_abundance/(3600*24)), abundance_Si, sigma_Si, fmt='o', capsize=5, label='Abundance with Error', color='blue')
axs[3].set_ylabel('Abundance_Si')
axs[3].set_xlabel('Time (days)')
axs[3].legend(loc='upper left')                    

axs[4].errorbar((time_abundance/(3600*24)), abundance_He, sigma_He, fmt='o', capsize=5, label='Abundance with Error', color='pink')
axs[4].set_ylabel('Abundance_He')
axs[4].set_xlabel('Time (days)')
axs[4].legend(loc='upper left')                    

for goes_file in goes_files:
    df = netCDF4.Dataset(os.path.join(data_dir, goes_file), mode='r')
    time_var = df.variables['time']
    dtime = netCDF4.num2date(time_var[:], time_var.units)
    goes_time = [(t - start_date_abundance).total_seconds() for t in dtime]
    axs[5].plot((np.array(goes_time)/(3600*24)), df.variables['xrsb_flux'][:], color='k', alpha=0.5, ls='--', label='GOES flux')

# Add labels and legends
axs[5].set_xlabel('Time (days)')
axs[5].set_ylim([0,5e-4])
plt.tight_layout()
#axs[5].legend(loc='upper right')
for ax in axs:
    ax.set_xlim([200,230])  # Replace xmin and xmax with the desired limits

plt.show()
